För att kunna köra inlärning med modifierad MNIST-kod är det nog lättast att lägga in kategorierna som nummer också i databasen. För att det ska vara lätt att mappa numren lägger jag in dem efter storleken på kategorierna.

In [105]:
import numpy as np
import sqlite3

In [46]:
conv = {0:lambda x: x.decode()} #Konvertern är för att filen är i unicode
categories = np.genfromtxt('../metadata/huvudgrupper.csv',  delimiter =";",dtype='U128',converters=conv)
print(categories[0])

['Likör' '3']


In [62]:
categories.T[1]


array(['3', '7', '28', '39', '6', '3', '33', '1', '9', '12', '23', '5',
       '31', '21', '18', '11', '16', '15', '5', '10', '24', '11', '4', '3',
       '1', '6', '3', '11', '51', '6', '15', '13', '94', '19', '11', '86',
       '22', '2', '2', '29', '8', '22', '25', '8', '14', '79', '8', '14',
       '12', '3', '11', '7', '4', '60', '22', '7', '15', '9', '31', '1',
       '1', '4', '6', '1', '12', '13', '3', '11', '3', '5', '4', '47',
       '80', '4', '39', '6', '42', '8', '2', '68', '3', '33', '18', '3',
       '3', '2', '26', '14', '9', '2', '7', '13', '28', '3', '5', '6', '2',
       '3', '25', '4', '1', '10', '9', '17', '25', '46', '42', '4', '127',
       '7', '55', '69', '2', '1', '2', '15', '1', '15'],
      dtype='<U128')

Eftersom dtype för categories är unicode går det inte att sortera categories direkt. Därför måste jag skapa en ny array "antal" som är index från 0 till antal kategorier i första kolumnen och antalet i varje kategori i den andra kolumnen.

Hade varit fint med ett sätt att slippa detta...

In [131]:
langd = len(categories.T[1])
antal = np.array([range(langd),categories.T[1]],dtype=int).T #Obs syntaxen med [] för att skapa arrayen
print(antal)


[[  0   3]
 [  1   7]
 [  2  28]
 [  3  39]
 [  4   6]
 [  5   3]
 [  6  33]
 [  7   1]
 [  8   9]
 [  9  12]
 [ 10  23]
 [ 11   5]
 [ 12  31]
 [ 13  21]
 [ 14  18]
 [ 15  11]
 [ 16  16]
 [ 17  15]
 [ 18   5]
 [ 19  10]
 [ 20  24]
 [ 21  11]
 [ 22   4]
 [ 23   3]
 [ 24   1]
 [ 25   6]
 [ 26   3]
 [ 27  11]
 [ 28  51]
 [ 29   6]
 [ 30  15]
 [ 31  13]
 [ 32  94]
 [ 33  19]
 [ 34  11]
 [ 35  86]
 [ 36  22]
 [ 37   2]
 [ 38   2]
 [ 39  29]
 [ 40   8]
 [ 41  22]
 [ 42  25]
 [ 43   8]
 [ 44  14]
 [ 45  79]
 [ 46   8]
 [ 47  14]
 [ 48  12]
 [ 49   3]
 [ 50  11]
 [ 51   7]
 [ 52   4]
 [ 53  60]
 [ 54  22]
 [ 55   7]
 [ 56  15]
 [ 57   9]
 [ 58  31]
 [ 59   1]
 [ 60   1]
 [ 61   4]
 [ 62   6]
 [ 63   1]
 [ 64  12]
 [ 65  13]
 [ 66   3]
 [ 67  11]
 [ 68   3]
 [ 69   5]
 [ 70   4]
 [ 71  47]
 [ 72  80]
 [ 73   4]
 [ 74  39]
 [ 75   6]
 [ 76  42]
 [ 77   8]
 [ 78   2]
 [ 79  68]
 [ 80   3]
 [ 81  33]
 [ 82  18]
 [ 83   3]
 [ 84   3]
 [ 85   2]
 [ 86  26]
 [ 87  14]
 [ 88   9]
 [ 89   2]
 [ 90   7]

In [132]:
#Sortering efter kolumn en med antal, med "fancy indexing"
antal = antal[antal[:,1].argsort()]

#Få arrayen i rätt ordning, eftersom man inte verkar kunna definiera omvänd sorteringsordning
#https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.flipud.html
antal = np.flipud(antal)

antal.T[0]

array([108,  32,  35,  72,  45, 111,  79,  53, 110,  28,  71, 105,  76,
       106,  74,   3,   6,  81,  58,  12,  39,   2,  92,  86,  98, 104,
        42,  20,  10,  54,  41,  36,  13,  33,  82,  14, 103,  16,  56,
        30,  17, 117, 115,  44,  87,  47,  91,  31,  65,  48,   9,  64,
        50,  34,  67,  15,  21,  27,  19, 101,  57,  88, 102,   8,  40,
        46,  77,  43,  51,  90,  55, 109,   1,  95,  29,  75,  25,  62,
         4,  18,  11,  94,  69,  52, 107,  61,  99,  70,  22,  73,   5,
        23,   0,  66,  83,  97,  68,  93,  80,  49,  26,  84, 114, 112,
        37,  78,  89,  38,  96,  85, 113,  24,   7,  63,  60,  59, 100, 116])

Den sorterade arrayen används sen som index till arrayen av kategorier...

Kan man verkligen inte fixa det här på ett bättre sätt?

In [108]:
ordered_categories = categories[antal.T[0]]
ordered_categories

array([['Kött färskt fryst tillagat', '127'],
       ['Grönsaker', '94'],
       ['Bullar kakor tårtor mm', '86'],
       ['Köttprodukter kötträtter', '80'],
       ['Sås dressing majonnäs', '79'],
       ['Fisk o skaldjursprodukter o rätter', '69'],
       ['Fisk färsk fryst kokt', '68'],
       ['Korv', '60'],
       ['Grönsaks- rotfrukts- baljväxträtter och produkter', '55'],
       ['Baljväxter (bönor, linser och ärter)', '51'],
       ['Soppa mat', '47'],
       ['Pizza paj pirog färdig smörgås', '46'],
       ['Mjukt bröd', '42'],
       ['Frukt färsk fryst', '42'],
       ['Hård matfettsblandning', '39'],
       ['Fågel', '39'],
       ['Potatisprodukter o rätter', '33'],
       ['Inälvor och organ', '33'],
       ['Potatis', '31'],
       ['Efterrätter', '31'],
       ['Mjöl stärkelse kli', '29'],
       ['Flingor - frukostflingor', '28'],
       ['Hårt bröd', '28'],
       ['Söta soppor kräm o efterrättssås', '26'],
       ['Kött processat', '25'],
       ['Ris risnudlar', '25

## Läs ner i databasen

Kolla om tabellen finns, och läs sen ner kategorinumren i databasen

In [107]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

columnObject = []

#Kolla om Huvudgrupp-kolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
newColumn = 'Huvudgrupp_n'
addColumn = True
for row in columns:
    print (row)
    if newColumn == row[1]:
        addColumn = False
        print (addColumn, newColumn)
if addColumn:
    #Man får se upp extra noga med kolumntypen, för den går inte att ändra i efterhand i sqlite...
    conn.execute('ALTER TABLE livs ADD COLUMN Huvudgrupp_n INTEGER;')
    conn.commit()

conn.close()

[0 'Livsmedelsnamn' 'TEXT' 0 None 0]
[1 'Livsmedelsnummer' 'TEXT' 0 None 0]
[2 'Energi_kcal' 'REAL' 0 None 0]
[3 'Energi_kJ' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[11 'Monosackarider_g' 'REAL' 0 None 0]
[12 'Disackarider_g' 'REAL' 0 None 0]
[13 'Sackaros_g' 'REAL' 0 None 0]
[14 'Fullkorn_totalt_g' 'REAL' 0 None 0]
[15 'Sockerarter_g' 'REAL' 0 None 0]
[16 'Summa_mattade_fettsyror_g' 'REAL' 0 None 0]
[17 'Fettsyra_4_0_10_0_g' 'REAL' 0 None 0]
[18 'Fettsyra_12_0_g' 'REAL' 0 None 0]
[19 'Fettsyra_14_0_g' 'REAL' 0 None 0]
[20 'Fettsyra_16_0_g' 'REAL' 0 None 0]
[21 'Fettsyra_18_0_g' 'REAL' 0 None 0]
[22 'Fettsyra_20_0_g' 'REAL' 0 None 0]
[23 'Summa_enkelomattade_fettsyror_g' 'REAL' 0 None 0]
[24 'Fettsyra_16_1_g' 'REAL' 0 None 0]
[25 'Fettsyra_18_1_g' 'REAL' 0 None 0]
[26 'Summa_fleromattade_fet

In [128]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

#print (len(ordered_categories))

i = 0
for row in ordered_categories:
    #print(row[1], row[0],i)
    #print (row[0].dtype)
    sql = 'UPDATE livs SET Huvudgrupp_n = '+str(i)+' WHERE Huvudgrupp = "'+row[0]+'";'
    print(sql)
    curs.execute(sql) #Kommentera bort denna vid debug...
    conn.commit()
    i+=1

conn.close()

UPDATE livs SET Huvudgrupp_n = 0 WHERE Huvudgrupp = "Kött färskt fryst tillagat";
UPDATE livs SET Huvudgrupp_n = 1 WHERE Huvudgrupp = "Grönsaker";
UPDATE livs SET Huvudgrupp_n = 2 WHERE Huvudgrupp = "Bullar kakor tårtor mm";
UPDATE livs SET Huvudgrupp_n = 3 WHERE Huvudgrupp = "Köttprodukter kötträtter";
UPDATE livs SET Huvudgrupp_n = 4 WHERE Huvudgrupp = "Sås dressing majonnäs";
UPDATE livs SET Huvudgrupp_n = 5 WHERE Huvudgrupp = "Fisk o skaldjursprodukter o rätter";
UPDATE livs SET Huvudgrupp_n = 6 WHERE Huvudgrupp = "Fisk färsk fryst kokt";
UPDATE livs SET Huvudgrupp_n = 7 WHERE Huvudgrupp = "Korv";
UPDATE livs SET Huvudgrupp_n = 8 WHERE Huvudgrupp = "Grönsaks- rotfrukts- baljväxträtter och produkter";
UPDATE livs SET Huvudgrupp_n = 9 WHERE Huvudgrupp = "Baljväxter (bönor, linser och ärter)";
UPDATE livs SET Huvudgrupp_n = 10 WHERE Huvudgrupp = "Soppa mat";
UPDATE livs SET Huvudgrupp_n = 11 WHERE Huvudgrupp = "Pizza paj pirog färdig smörgås";
UPDATE livs SET Huvudgrupp_n = 12 WHERE H